<a href="https://colab.research.google.com/github/sam80FDS/Sam/blob/main/Master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
#  HEADLESS STUDIO v30.0 (The AudioCraft Fix)
#  Engine: ComfyUI + GitMyLo + ParlerTTS
#  Status: FIXED (Version conflicts resolved via --no-deps)
# ==============================================================================

import os
import time
import subprocess

# ------------------------------------------------------------------------------
# STEP 1: SYSTEM LAYER (FFMPEG & DRIVERS)
# ------------------------------------------------------------------------------
print("🛠️ STEP 1/6: PREPARING SYSTEM...")
# Clean up previous installs to ensure a fresh start
!rm -rf /content/ComfyUI
!apt-get update -qq
!apt-get install -y ffmpeg libsndfile1 pkg-config -qq

# ------------------------------------------------------------------------------
# STEP 2: INSTALL COMFYUI FRAMEWORK
# ------------------------------------------------------------------------------
print("🚀 STEP 2/6: INSTALLING COMFYUI...")
%cd /content
!git clone https://github.com/comfyanonymous/ComfyUI
%cd /content/ComfyUI
!pip install -r requirements.txt --quiet

# ------------------------------------------------------------------------------
# STEP 3: INSTALL CUSTOM NODES
# ------------------------------------------------------------------------------
print("📦 STEP 3/6: DOWNLOADING NODES...")
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
!git clone https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite.git
!git clone https://github.com/cubiq/ComfyUI_IPAdapter_plus.git

# Audio Nodes (We clone them, but install requirements manually in Step 4)
!git clone https://github.com/gitmylo/ComfyUI-audio-nodes.git
!git clone https://github.com/smthemex/ComfyUI_ParlerTTS.git

# ------------------------------------------------------------------------------
# STEP 4: DEPENDENCY SURGERY (THE CRITICAL FIX)
# ------------------------------------------------------------------------------
print("❤️‍🩹 STEP 4/6: INSTALLING AUDIO ENGINES (NO-DEPS MODE)...")
%cd /content

# 1. Install AudioCraft WITHOUT destroying Colab's PyTorch (GPU Drivers)
!pip install --no-deps git+https://github.com/facebookresearch/audiocraft.git

# 2. Manually install ONLY the safe missing libraries
# We purposely exclude 'torch' so Colab keeps its native GPU version.
!pip install soundfile librosa encodec funcy omegaconf hydra-core protobuf sentencepiece transformers --quiet

# 3. Install AudioTools (Required for ParlerTTS to work)
!pip install git+https://github.com/descriptinc/audiotools.git

# 4. Final Polish
!pip install scipy --upgrade --quiet

# ------------------------------------------------------------------------------
# STEP 5: DOWNLOAD AI MODELS
# ------------------------------------------------------------------------------
print("💾 STEP 5/6: DOWNLOADING MODELS (This may take a minute)...")

# 1. Image & Video Models
!wget -nc -O /content/ComfyUI/models/checkpoints/dreamshaper_8.safetensors https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaper_8_pruned.safetensors
!wget -nc -O /content/ComfyUI/models/checkpoints/svd_xt.safetensors https://huggingface.co/stabilityai/stable-video-diffusion-img2vid-xt/resolve/main/svd_xt.safetensors

# 2. Audio Models (MusicGen Small)
!wget -nc -O /content/ComfyUI/models/checkpoints/musicgen_small.safetensors https://huggingface.co/facebook/musicgen-small/resolve/main/model.safetensors

# 3. TTS Models (Parler Mini)
!mkdir -p /content/ComfyUI/models/parler_tts
!wget -nc -O /content/ComfyUI/models/parler_tts/parler-tts-mini-v1.safetensors https://huggingface.co/parler-tts/parler-tts-mini-v1/resolve/main/model.safetensors

# 4. Helper Models (Upscalers & Frame Interpolation)
!mkdir -p /content/ComfyUI/models/upscale_models
!wget -nc -O /content/ComfyUI/models/upscale_models/RealESRGAN_x4plus.pth https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth
!mkdir -p /content/ComfyUI/models/vfi
!wget -nc -O /content/ComfyUI/models/vfi/rife47.pth https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/rife47.pth

# ------------------------------------------------------------------------------
# STEP 6: LAUNCH SERVER
# ------------------------------------------------------------------------------
print("🌩️ STEP 6/6: INITIALIZING CLOUDFLARE TUNNEL...")
%cd /content/ComfyUI
!wget -nc https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

print("----------------------------------------------------------------")
print("✅ INSTALL COMPLETE. INITIALIZING SERVER...")
print("⚠️ COPY THE LINK ENDING IN 'trycloudflare.com' INTO N8N")
print("----------------------------------------------------------------")

!python main.py --listen --dont-print-server & cloudflared tunnel --url http://localhost:8188
z